In this notebook we will add  realistic noise effect to images based on a camera nooise model. We follow the blog example at (http://kmdouglass.github.io/posts/modeling-noise-for-image-simulations/ ).

First we import all necessary packages and set camera constants.
These values depend on the camera and in our case we assume an OV2640 CameraChip(tm).
Other cameras will have different values for the constants below.

In [ ]:
import glob
import os
from skimage import color

import OpenEXR
import numpy as np
from Imath import PixelType
from PIL import Image

# which file we want to process
FILE_ENDING = ".png"

# CAMERA CONSTANTS

# the number of always activated sensors for each pixel
# this information was not given, just assumed
BASELINE_CAMERA = 0

# the amount of current per activated sensor
SENSITIVITY_CAMERA = 0.6

# the color depth for each pixel
BIT_DEPTH_CAMERA = 8

# the always active current inside a sensor
DARK_NOISE_CAMERA = 0.015

# the percentage of electrons activating the sensor
# assumed to be the same as the camera in the blog post
QUANTUM_EFFICIENCY_CAMERA = 0.69

We take an 2D array of irradiance values (number of electrons hitting the sensors), simulate a poisson process for the values and add the dark current noise.

In [ ]:
# Function to add camera noise
def add_camera_noise(input_irrad_photons,
                     qe=QUANTUM_EFFICIENCY_CAMERA,
                     sensitivity=SENSITIVITY_CAMERA,
                     dark_noise=DARK_NOISE_CAMERA,
                     bitdepth=BIT_DEPTH_CAMERA,
                     baseline=BASELINE_CAMERA,
                     rs=np.random.RandomState(seed=1234)):

    # Add shot noise
    # For each pixel, simulate a poission distribution
    photons = rs.poisson(input_irrad_photons, size=input_irrad_photons.shape)

    # Convert to electrons, since only a percentage of electrons activate the sensors
    electrons = qe * photons

    # simulate a normal distributed dark current centered around 0 for each pixel
    dark_current = rs.normal(scale=dark_noise, size=electrons.shape)

    # add both together to get the number of electrons per pixel
    electrons_out = dark_current + electrons

    max_adu = np.int(2 ** bitdepth - 1)

    # each electron
    adu = (electrons_out * sensitivity).astype(np.int)
    adu += baseline
    adu[adu > max_adu] = max_adu  # models pixel saturation

    return adu

Now we apply the function above to our images. We also need some helper functions to convert images from OpenEXR and save them onto the disk.


In [ ]:
# converts the openexr float values into the standard  [0, 255] range for colors.
# see https://blender.stackexchange.com/questions/65288/convert-openexr-float-to-color-value
def norm(val):
    return 255 * (val * 12.92 if val <= 0.0031308 else 1.055 * val ** (1.0 / 2.4) - 0.055)
norm = np.vectorize(norm) # for easier parallelization

# a helper function to save an image
def save_image(pixel_data, filename):
    img = Image.fromarray(pixel_data.astype(np.uint8), 'RGB')
    img.save(filename)


def convert_from_exr(file_path):
    image = OpenEXR.InputFile(file_path)
    W = image.header()["displayWindow"].max.x + 1
    H = image.header()["displayWindow"].max.y + 1
    pixels = np.zeros((H, W, 3))
    datatype = np.float32
    if image.header()["channels"]["R"].type.v == PixelType.HALF:
        datatype = np.float16
    pixels[:, :, 0] = np.frombuffer(image.channel('R'), dtype=datatype).reshape((H, W))
    pixels[:, :, 1] = np.frombuffer(image.channel('G'), dtype=datatype).reshape((H, W))
    pixels[:, :, 2] = np.frombuffer(image.channel('B'), dtype=datatype).reshape((H, W))
    pixels = np.clip(pixels, 0, 1)
    pixels = norm(pixels)
    return pixels

def convert_from_png(file_path):
    image = Image.open(file_path)
    pixels = list(image.getdata())
    width, height = image.size
    pixels = np.array([pixels[i * width:(i + 1) * width] for i in range(height)])
    return pixels

def add_noise_to_image(file_path):
    # this part is to convert OpenEXR images from their format into a 2D-Array of RGB values from 0-255.OpenEXR
    # If other image formats are used, this is not be necessary
    # In that case, use the function convert_from_png or write a similar function for other image formats
    #pixel_data = convert_from_exr(file_path)
    pixel_data = convert_from_png(file_path)

    # calculate number of electrons hitting the sensor and apply noise model
    number_of_electrons_per_pixel = pixel_data / QUANTUM_EFFICIENCY_CAMERA / SENSITIVITY_CAMERA
    noisy_pixels = add_camera_noise(number_of_electrons_per_pixel)

    # save the clean and the noisy image as PNG next to the original version
    clean_file_name = file_path.replace(FILE_ENDING, "_clean.png")
    save_image(pixel_data, clean_file_name)

    noisy_file_name = file_path.replace(FILE_ENDING, "_noisy.png")
    save_image(noisy_pixels, noisy_file_name)

# we take every image in the current directory ending with ".exr" and apply our noise model
for file in glob.glob(f"*{FILE_ENDING}"):
    print(file)
    add_noise_to_image(file_path=file)
print("EMD")